In [1]:
import os
from keras.preprocessing import image
import matplotlib.pyplot as plt 
import numpy as np
from keras.utils import to_categorical
import random,shutil
from keras.models import Sequential
from keras.layers import Dropout,Conv2D,Flatten,Dense, MaxPooling2D, BatchNormalization
from keras.models import load_model

In [2]:
def generator(dir, gen=image.ImageDataGenerator(rescale=1./255), shuffle=True,batch_size=1,target_size=(24,24),class_mode='categorical' ):

    return gen.flow_from_directory(dir,batch_size=batch_size,shuffle=shuffle,color_mode='grayscale',class_mode=class_mode,target_size=target_size)

In [3]:
BS= 32
TS=(24,24)
train_batch= generator('data/train',shuffle=True, batch_size=BS,target_size=TS)
valid_batch= generator('data/valid',shuffle=True, batch_size=BS,target_size=TS)
SPE= len(train_batch.classes)//BS
VS = len(valid_batch.classes)//BS
print(SPE,VS)

Found 1234 images belonging to 2 classes.
Found 218 images belonging to 2 classes.
38 6


In [4]:
model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(24,24,1)),
    MaxPooling2D(pool_size=(1,1)),
    Conv2D(32,(3,3),activation='relu'),
    MaxPooling2D(pool_size=(1,1)),
#32 convolution filters used each of size 3x3
#again
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(1,1)),

#64 convolution filters used each of size 3x3
#choose the best features via pooling
    
#randomly turn neurons on and off to improve convergence
    Dropout(0.25),
#flatten since too many dimensions, we only want a classification output
    Flatten(),
#fully connected to get all relevant data
    Dense(128, activation='relu'),
#one more dropout for convergence' sake :) 
    Dropout(0.5),
#output a softmax to squash the matrix into output probabilities
    Dense(2, activation='softmax')
])

In [5]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)

model.save('models/cnnCat2.h5', overwrite=True)

C:\Users\kavya\AppData\Local\Temp\ipykernel_15688\814966032.py:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_batch, validation_data=valid_batch,epochs=15,steps_per_epoch=SPE ,validation_steps=VS)


Epoch 1/15


38/38 [==============================] - 42s 1s/step - loss: 0.4491 - accuracy: 0.7970 - val_loss: 0.2442 - val_accuracy: 0.9219
Epoch 2/15
38/38 [==============================] - 11s 280ms/step - loss: 0.1916 - accuracy: 0.9243 - val_loss: 0.1533 - val_accuracy: 0.9375
Epoch 3/15
38/38 [==============================] - 12s 309ms/step - loss: 0.1343 - accuracy: 0.9476 - val_loss: 0.1496 - val_accuracy: 0.9583
Epoch 4/15
38/38 [==============================] - 12s 320ms/step - loss: 0.1176 - accuracy: 0.9601 - val_loss: 0.1163 - val_accuracy: 0.9583
Epoch 5/15
38/38 [==============================] - 12s 310ms/step - loss: 0.0952 - accuracy: 0.9626 - val_loss: 0.1260 - val_accuracy: 0.9479
Epoch 6/15
38/38 [==============================] - 12s 314ms/step - loss: 0.0792 - accuracy: 0.9684 - val_loss: 0.1164 - val_accuracy: 0.9427
Epoch 7/15
38/38 [==============================] - 17s 458ms/step - loss: 0.0728 - accuracy: 0.9709 - val_loss: 0.1533 - val_accuracy: 0.9375


C:\Users\kavya\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
